# Image Deraining using Convolutional Neural Networks

<img src="https://raw.githubusercontent.com/andywang947/Common/main/9.PNG" width="500px">

**Image credit: Rain100L Dataset**  

After this homework, you would ideally have learned:  

- To create a simple solution that is easy to set up.
- Learning about the principles of image denoising.
- To manage the image data and apply augmentation techniques to the images.
- To train and optimize the model for better performance.
- To develop strategies that can help navigate through various options to find the best solution.

## Introduction

Low-level tasks in computer vision, like color enhancement, denoising, dehazing, are a very important and fundamental step in computer vision. Correcting a degraded image through image processing methods is crucial for subsequent computer vision tasks. For example, in **Assignment 3**, even the results obtained during training were excellent, applying it in reality scenarios with degradation images would significantly decrease the performance.

In this assignment, you will attempt to use a CNN for the task of rain removal, which we ofter call it **deraining** task. The presence of rain in an image poses a challenge to machine vision, similar to the difficulty faced when driving with a broken windshield wiper.

## Homework

In this assignment, we will train a CNN-based image deraining network using a dataset called Rain100L, which contains 200 training images and 100 testing images. The goal of this assignment is to train an image deraining network using CNN and to improve the model through various experiments. Please follow the instructions below to build the image deraining network.

## Get the dataset

Like the previous work, we need to connect with your google drive to get data for this training.

**First, create share folder's shortcut to your google drive.**  
1. Go google drive website and login  
2. Go to https://drive.google.com/drive/folders/1D1A_Rx5LSDxW0VSswwM1V1bGvibOho8_?usp=sharing
3. Create shared folder's shortcut on your dirve  
<img src="https://github.com/andywang947/Common/blob/main/10.PNG?raw=true" width="800px">  
<img src="https://github.com/andywang947/Common/blob/main/11.PNG?raw=true" width="800px">  

4. You'll see the folder is on your drive now!  
<img src="https://github.com/andywang947/Common/blob/main/12.PNG?raw=true" width="800px">  

5. Done!

Next, let's check if you have correctly obtained the files.

(optional: We recommend you use T4 GPU to do the training. You can check the assignment 3 for the detail.)

First, connect to your google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
file_path = "/content/drive/MyDrive/HW4_revised/HW4_image_deraining.zip"
if os.path.exists(file_path):
  print("You got the file !")
else :
  print("You didn't obtain the files now, need to correct your path.")

If you check you got the file correctly, next, we can unzip the files.

In [ ]:
! unzip /content/drive/MyDrive/HW4_revised/HW4_image_deraining.zip -d /content/

In [ ]:
import os
def check_data(data_path) :
  if os.path.exists(data_path) :
    files = os.listdir(data_path)
    print("There are ",len(files),"images in the dictionary:",data_path)
  else :
    print("The dictionary path is wrong ! You need to fix your dictionary path !")

train_input_dir = "/content/Rain100L/train/input"
check_data(train_input_dir)
train_target_dir = "/content/Rain100L/train/target"
check_data(train_target_dir)
test_input_dir = "/content/Rain100L/test/input"
check_data(test_input_dir)
test_target_dir = "/content/Rain100L/test/target"
check_data(test_target_dir)

You should see you have 200,200,100,100 images in the dictionaries.

## Dataloader

Now, we have all data correctly. Next, we can start our training process.

First, we need to load our data.

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PairedImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_filenames = [os.path.join(input_dir, f) for f in sorted(os.listdir(input_dir))]
        self.target_filenames = [os.path.join(target_dir, f) for f in sorted(os.listdir(target_dir))]
        self.transform = transform

    def __len__(self):
        return len(self.input_filenames)

    def __getitem__(self, idx):
        input_image = Image.open(self.input_filenames[idx]).convert('RGB')
        target_image = Image.open(self.target_filenames[idx]).convert('RGB')

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])

# Dataset
train_dataset = PairedImageDataset(train_input_dir, train_target_dir, transform=transform)
test_dataset = PairedImageDataset(test_input_dir, test_target_dir, transform=transform)

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)


# Fetch a batch and display the images
data_iter = iter(train_dataloader)
images, targets = next(data_iter)

# Display images using matplotlib
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i in range(4):
    img_input = images[i].numpy().transpose((1, 2, 0))
    img_input = img_input.clip(0, 1)

    img_target = targets[i].numpy().transpose((1, 2, 0))
    img_target = img_target.clip(0, 1)

    ax_input = axes[0, i]
    ax_input.imshow(img_input)
    ax_input.axis('off')
    ax_input.set_title(f'Input {i+1}')

    ax_target = axes[1, i]
    ax_target.imshow(img_target)
    ax_target.axis('off')
    ax_target.set_title(f'Target {i+1}')

plt.show()

Now, we know we have our dataloader, which can feed the images to our model.
And by showing the data, you can see our input data and the target data.

## Model

Next, let's build our CNN model.
Here I use the Deep, narrow, CNN as the model here.

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary


class DeepNarrowCNN(nn.Module):
    def __init__(self):
        super(DeepNarrowCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = DeepNarrowCNN()
# if you use GPU, you can speed your training time.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

In [ ]:
summary(model, (3, 112, 112))

By the summary of our model, we can see the structure of our model, and the number of parameters. These are important reference materials for analyzing the performance and quality of the model.

Here, we have 47283 parameters.

## Training

Now, we can start our training.
Here, I set the training process to do 100 epochs, if epochs bigger than 50 and loss didn't decrease in 10 epochs, then early stopping the training.

In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch

def unnormalize(image):
    image = image.numpy().transpose((1, 2, 0))
    image = np.clip(image, 0, 1)
    return image

def testing(model,test_dataloader):
  model.eval()
  model.to('cpu')

  data_iter = iter(test_dataloader)
  inputs, targets = next(data_iter)

  with torch.no_grad():
      predictions = model(inputs)

  inputs = inputs.cpu()
  targets = targets.cpu()
  predictions = predictions.cpu()

  fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 9))
  for i in range(4):
      ax = axes[0, i]
      ax.imshow(unnormalize(inputs[i]))
      ax.axis('off')
      ax.set_title('Input Image')

      ax = axes[1, i]
      ax.imshow(unnormalize(predictions[i]))
      ax.axis('off')
      ax.set_title('Predicted Image')

      ax = axes[2, i]
      ax.imshow(unnormalize(targets[i]))
      ax.axis('off')
      ax.set_title('Target Image')

  plt.show()

def training(criterion,optimizer,model):

  num_epochs = 100
  testing_epoch = 10
  patience = 10  # For Early stopping use
  best_loss = float('inf')
  epochs_no_improve = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f'Using device: {device}')
  model = model.to(device)

  for epoch in range(num_epochs):
      running_loss = 0.0
      for inputs, targets in train_dataloader:
          inputs, targets = inputs.to(device), targets.to(device)
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, targets)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      epoch_loss = running_loss / len(train_dataloader)
      print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

      if epoch % testing_epoch == 0 :
        testing(model,test_dataloader)
        model = model.to(device)

      if epoch > 50 :
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

For the first model, here we use the previous DeepNarrowCNN as the baseline model, and L1 loss, and NAdam as the optimizer.
Let's do the training to see the results.

In [ ]:
model = DeepNarrowCNN()
criterion = nn.L1Loss()
optimizer = optim.NAdam(model.parameters(), lr=0.001)
training(criterion,optimizer,model)

## Evaluation

After the training, we can test our model by evaluation.
First, let's see our model outputs, if they satisfy our task?

In [ ]:
testing(model,test_dataloader)

It's not accurate to rely solely on our eyes, so here we use **PSNR(Peak signal-to-noise ratio)** and **SSIM(structural similarity)** metrics to evaluate the performance of the rain removal, which are common metrics in low-level vision tasks.

In [ ]:
import torch
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np

def calculate_psnr_ssim(inputs, predictions, targets):
    psnr_values = []
    ssim_values = []

    inputs_np = inputs.numpy().transpose((0, 2, 3, 1))
    predictions_np = predictions.numpy().transpose((0, 2, 3, 1))
    targets_np = targets.numpy().transpose((0, 2, 3, 1))

    predictions_np = np.clip(predictions_np, 0, 1)
    targets_np = np.clip(targets_np, 0, 1)

    for i in range(inputs.shape[0]):
        psnr_val = psnr(targets_np[i], predictions_np[i], data_range=1)
        ssim_val, _ = ssim(targets_np[i], predictions_np[i], data_range=1, channel_axis=-1, full=True)

        psnr_values.append(psnr_val)
        ssim_values.append(ssim_val)

    return np.mean(psnr_values), np.mean(ssim_values)

def evaluation(model,test_dataloader) :
  model.eval()
  model.to('cpu')

  total_psnr = 0
  total_ssim = 0
  count = 0
  all_predictions = []

  for inputs, targets in test_dataloader:
      with torch.no_grad():
          predictions = model(inputs)

      inputs = inputs.cpu()
      predictions = predictions.cpu()
      all_predictions.append(predictions)
      targets = targets.cpu()

      psnr_val, ssim_val = calculate_psnr_ssim(inputs, predictions, targets)
      total_psnr += psnr_val
      total_ssim += ssim_val
      count += 1

  all_predictions = np.concatenate(all_predictions, axis=0)
  average_psnr = total_psnr / count
  average_ssim = total_ssim / count

  print(f'Average PSNR: {average_psnr:.2f}')
  print(f'Average SSIM: {average_ssim:.2f}')

In [ ]:
evaluation(model,test_dataloader)

## Ways to improve

Now, we have a baseline model! This is often the first step in the training process, and then we need to try to improve the model for better performance!

### Optimizer

First of all, optimizer, in the previous model, we use NAdam as our optimizer. Now, let's change the optimizer to Adam, which is also a gradient descent way to optimize the parameters.

In [ ]:
model = DeepNarrowCNN()
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Change the optimizer to Adam
training(criterion,optimizer,model)

In [ ]:
testing(model,test_dataloader)

In [ ]:
evaluation(model,test_dataloader)

### Loss function

Another way to improve is about the choice of loss function.

Above we use L1 loss, which is the absolute distance between the output and the target.

Next, let's try MSE (Mean Square Error) as the loss function.

In [ ]:
model = DeepNarrowCNN()
criterion = nn.MSELoss()  # Use MSE as the loss function.
optimizer = optim.Adam(model.parameters(), lr=0.001)
training(criterion,optimizer,model)

In [ ]:
testing(model,test_dataloader)

In [ ]:
evaluation(model,test_dataloader)

### The structure of the model

Another way to improve is to change the stucture of the model,

In [ ]:
import torch
import torch.nn as nn

class WideCNN(nn.Module):
    def __init__(self):
        super(WideCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = WideCNN() # Our new model, with the classical encoder-decoder structure
model = model.to(device)

In [ ]:
summary(model, (3, 112, 112))

By the summary, we can see the model parameters are still about the same as the previous DeepNarrowCNN, but the strucutre design is completly different. Next, let's train the model and see the results.

In [ ]:
model = WideCNN() # Our new model, with the classical encoder-decoder structure
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
training(criterion,optimizer,model)

In [ ]:
testing(model,test_dataloader)

In [ ]:
evaluation(model,test_dataloader)

 After using this classic encoder-decoder architecture, you should see a significant improvement in the results! This proves that in the world of deep learning, simply increasing the complexity of the model does not always lead to better results. Smart design can improve performance while maintaining the number of parameters!

### Data augmentation

Again, after the assignment 3, we see the signigicant improve after data augmentation through we don't have so many images in our dataset.

Here, we can also try two different ways to improve.

We need to add more images to the new traininng dataset.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

old_train_dir = "/content/Rain100L/train"
new_train_dir = "/content/Rain100L/newtrain"

if not os.path.exists(new_train_dir):
    os.makedirs(new_train_dir)
    print(f"Directory '{new_train_dir}' created.")
    os.makedirs(os.path.join(new_train_dir, "input"))
    os.makedirs(os.path.join(new_train_dir, "target"))
else:
    print(f"Directory '{new_train_dir}' already exists.")

old_input_dir = os.path.join(old_train_dir, "input")
old_target_dir = os.path.join(old_train_dir, "target")

new_input_dir = os.path.join(new_train_dir, "input")
new_target_dir = os.path.join(new_train_dir, "target")

input_filenames = [os.path.join(old_input_dir, f) for f in sorted(os.listdir(old_input_dir))]
target_filenames = [os.path.join(old_target_dir, f) for f in sorted(os.listdir(old_target_dir))]

base_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

for i, (input_filename, target_filename) in enumerate(zip(input_filenames, target_filenames)):
    original_input_image = Image.open(input_filename).convert('RGB')
    original_target_image = Image.open(target_filename).convert('RGB')

    flipped_input_image = transforms.functional.vflip(original_input_image)
    flipped_target_image = transforms.functional.vflip(original_target_image)

    angle = random.uniform(-15, 15)
    rotated_input_image = transforms.functional.rotate(original_input_image, angle)
    rotated_target_image = transforms.functional.rotate(original_target_image, angle)

    original_input_image.save(os.path.join(new_input_dir, f"original_input_{i}.png"))
    original_target_image.save(os.path.join(new_target_dir, f"original_target_{i}.png"))

    flipped_input_image.save(os.path.join(new_input_dir, f"flipped_input_{i}.png"))
    flipped_target_image.save(os.path.join(new_target_dir, f"flipped_target_{i}.png"))

    rotated_input_image.save(os.path.join(new_input_dir, f"rotated_input_{i}.png"))
    rotated_target_image.save(os.path.join(new_target_dir, f"rotated_target_{i}.png"))


In [ ]:
import os
def check_data(data_path) :
  if os.path.exists(data_path) :
    files = os.listdir(data_path)
    print("There are ",len(files),"images in the dictionary:",data_path)
  else :
    print("The dictionary path is wrong ! You need to fix your dictionary path !")

train_input_dir = new_input_dir
check_data(train_input_dir)
train_target_dir = new_target_dir
check_data(train_target_dir)

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PairedImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_filenames = [os.path.join(input_dir, f) for f in sorted(os.listdir(input_dir))]
        self.target_filenames = [os.path.join(target_dir, f) for f in sorted(os.listdir(target_dir))]
        self.transform = transform

    def __len__(self):
        return len(self.input_filenames)

    def __getitem__(self, idx):
        input_image = Image.open(self.input_filenames[idx]).convert('RGB')
        target_image = Image.open(self.target_filenames[idx]).convert('RGB')

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])

# Dataset
train_dataset = PairedImageDataset(new_input_dir, new_target_dir, transform=transform)
test_dataset = PairedImageDataset(test_input_dir, test_target_dir, transform=transform)

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)


# Fetch a batch and display the images
data_iter = iter(train_dataloader)
images, targets = next(data_iter)

# Display images using matplotlib
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i in range(4):
    img_input = images[i].numpy().transpose((1, 2, 0))
    img_input = img_input.clip(0, 1)

    img_target = targets[i].numpy().transpose((1, 2, 0))
    img_target = img_target.clip(0, 1)

    ax_input = axes[0, i]
    ax_input.imshow(img_input)
    ax_input.axis('off')
    ax_input.set_title(f'Input {i+1}')

    ax_target = axes[1, i]
    ax_target.imshow(img_target)
    ax_target.axis('off')
    ax_target.set_title(f'Target {i+1}')

plt.show()

In [ ]:
model = WideCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
training(criterion,optimizer,model)

In [ ]:
testing(model,test_dataloader)

In [ ]:
evaluation(model,test_dataloader)

### Add more parameters

Besides change the structure of the model, we can still add more layers to the model.

But since if we add more parameters to the model, it costs more time to the training process. So through it's an easy way to get better performance, but here we try this improve in the end.

In [ ]:
class DeepWideCNN(nn.Module):
    def __init__(self):
        super(DeepWideCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = DeepWideCNN()
criterion = nn.MSELoss()  # Use MSE as the loss function.
optimizer = optim.Adam(model.parameters(), lr=0.001)
training(criterion,optimizer,model)

In [ ]:
testing(model,test_dataloader)

In [ ]:
evaluation(model,test_dataloader)

If our environments are the same, you should be able to see that one of the PSNR values in the above settings exceeds 30! This process is a classic deep learning approach to problem-solving. Hope you enjoy this deep learning journey!

## Problems

1. The major challenge in addressing the deraining problem, as compared to issues like image classification and facial recognition, is forming paired datasets. It's easy to collect scenes with rain, but obtaining corresponding clean images is extremely difficult. Consequently, many synthetic datasets exist today, such as the rain100L used in this assignment. What impact do you think using synthetic datasets might have on training results, and how can we solve this problem? (Maybe we don't need paired data?) Please provide one idea.

  You may find some interesting ideas in the following papers.

  Noise2Noise: Learning Image Restoration without Clean Data
  https://arxiv.org/abs/1803.04189

  Image Deraining via Self-supervised Reinforcement Learning
  https://arxiv.org/abs/2403.18270

2. Deep learning implementations are always based on our observations to gradually improve the model. In the previous process, you might have noticed that although the deraining results seem quite good, the colors of the images may differ from the original ones. Please propose two ideas (no need to implement) on how we can improve the model based on the observation that **'the colors of the derained images differ from the input images.'** These ideas can be in any aspect such as the model, loss function, data preprocessing, data augmentation,or the whole training framework,etc.

3. Many tasks in image processing and computer vision require separate models, each trained for a specific purpose, such as deraining, denoising, deblurring, and super-resolution. However, deploying multiple models in practical applications can be resource-intensive and inefficient. Please propose one idea on how we can develop an all-in-one model that can handle multiple image restoration tasks simultaneously.

  You may find some interesting ideas in the following papers.

  Restormer: Efficient Transformer for High-Resolution Image Restoration
  https://arxiv.org/abs/2111.09881

  Language-driven All-in-one Adverse Weather Removal
  https://arxiv.org/abs/2312.01381